In [2]:
!wget https://github.com/alexmk7/nlp-2024/raw/refs/heads/main/data/news.txt.gz

--2025-01-08 14:16:56--  https://github.com/alexmk7/nlp-2024/raw/refs/heads/main/data/news.txt.gz
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexmk7/nlp-2024/refs/heads/main/data/news.txt.gz [following]
--2025-01-08 14:16:56--  https://raw.githubusercontent.com/alexmk7/nlp-2024/refs/heads/main/data/news.txt.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7700055 (7.3M) [application/octet-stream]
Saving to: ‘news.txt.gz’

news.txt.gz         100%[===================>]   7.34M  --.-KB/s    in 0.07s   

2025-01-08 14:16:56 (98.0 MB/s) - ‘news.txt.gz’ saved [7700055/7700055]



In [11]:
import gzip
import pandas as pd

In [12]:
with gzip.open("news.txt.gz", "rt", encoding="utf-8") as f:
    lines = []
    for line in f:
        lines.append(line.strip().split("\t"))

In [13]:
df = pd.DataFrame(lines, columns=["Label", "Title", "Text"])

# Парсинг

In [14]:
!pip install yargy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=63ad385724cc7b97cb2eb05a94c5bce3f2c69b5b0638fa68437186998fe6ea30
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [200]:
# df[df.apply(lambda x: "родился" in x["Text"].split(" "), axis=1)]
sample = df.loc[df.apply(lambda x: "родился" in x["Text"].split(" "), axis=1)]["Text"].values
sample

array(['Американский актер Стивен Хилл скончался в Нью-Йорке в возрасте 94 лет, сообщает во вторник, 23 августа, Deadline. Он известен ролью окружного прокурора Адама Шиффа в сериале «Закон и порядок».Хилл также исполнил главную роль в телесериале «Миссия невыполнима», выходившем с 1966 по 1973 год.Актер родился в Сиэтле, в семье еврейских эмигрантов, выходцев из России. Его настоящее имя Соломон, а фамилия — Краковский (по другим сведениям — Берг). Он начал артистическую карьеру на Бродвее в середине 1960-х. В спектакле «Рождение флага» он играл вместе с Марлоном Брандо.После закрытия проекта «Миссия невыполнима» Хилл около десяти лет не снимался и не играл в театре. Он вернулся в кино в начале 1980-х, сыграв, среди прочего, в боевике «Без компромиссов» с Арнольдом Шварценеггером.За роль в сериале «Закон и порядок» Хилл был номинирован на премию «Эмми» в 1997 году.',
       'Президент России Владимир Путин назначил руководителем главка МВД России по борьбе с коррупцией генерал-майора 

In [290]:
# Имя
gnc = gnc_relation()

Name = fact(
    'Name',
    ['first', 'last']
)

FIRST = gram('Name').interpretation(
    Name.first.inflected()
).match(gnc)

LAST = gram('Surn').interpretation(
    Name.last.inflected()
).match(gnc)

NAME = or_(
    rule(
        FIRST
    ),
    rule(
        LAST
    ),
    rule(
        FIRST,
        LAST
    ),
    rule(
        LAST,
        FIRST
    )
).interpretation(
    Name
)

# Дата рождения
Date = fact(
    'Date',
    ['year', 'month', 'day']
)

class Date(Date):
    @property
    def as_datetime(self):
        return date(self.year, self.month, self.day)


MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}


DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month.custom(int)
)

YEAR = and_(
    gte(1),
    lte(2024)
).interpretation(
    Date.year.custom(int)
)

MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.__getitem__)
)

YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
)

DATE = or_(
    rule(YEAR, '-', MONTH, '-', DAY),
    rule(DAY, '.', MONTH, '.', YEAR),
    rule(
        gram("PREP").optional(),
        DAY.optional(),
        MONTH_NAME.optional(),
        YEAR,
        YEAR_WORDS.optional()
    )
).interpretation(Date)

# Локация
Location = fact(
    'Location',
    ['location']
)

LOCATION = rule(
    gram('PREP'),
    gram('NOUN').optional(),
    and_(
        is_capitalized(),
        gram('NOUN')
    ).interpretation(Location.location.inflected())
).interpretation(Location)

# Общее инфо
Person = fact(
    'Person',
    ['name', 'date', 'location']
)

PERSON = or_(
    rule(
        NAME.interpretation(Person.name),
        gram("VERB").optional(),
        DATE.interpretation(Person.date),
        LOCATION.interpretation(Person.location),
    ),
    rule(
        NAME.interpretation(Person.name),
        gram("VERB").optional(),
        LOCATION.interpretation(Person.location),
        DATE.interpretation(Person.date),
    ),
).interpretation(Person)

parser = Parser(PERSON)

In [300]:
# Дмитрий Чернявский родился в Артемовске 5 марта 1992 года
# Эли Визель родился в 1928
# Юрий Яковлев родился в Москве в 1928 году
# Смирнов родился 18 декабря 1937 года
# Патрик родился в Бронксе в 1990 году
# Николай Караченцов родился 27 октября 1944 года в Москве
# Орест I (он же Орест Федорович Карелин-Романишин-Русин) родился в 1950 году во Львове
# Сергей Довлатов родился в 1941 году
# Головко родился 24 ноября 1932 года в городе Бар
# Хуан Жоу родился в 1979 году
# Джордж родился 22 июля 2013-го
# Том Роу (полное имя Томас Джон Роу - Thomas John Rowe) родился в 1956 году
# Игорь Доценко родился в 1953 году в Хмельницкой области Украины
# Андрей Сердюков родился в 1962 году
# Анатолий Сморгонский родился в 1979 году
# Энгельбарт родился в 1925 году в США

test_sample = {
    df[df["Text"].str.find("Дмитрий Чернявский родился в Артемовске 5 марта 1992 года") > 0]["Text"].values[0]: Person(name=Name(first='дмитрий', last='чернявский'), date=Date(year=1992, month=3, day=5), location=Location(location='артемовск')),
    df[df["Text"].str.find("Эли Визель родился в 1928") > 0]["Text"].values[0]: Person(name=Name(first='эли', last='визель'), date=Date(year=1928, month=None, day=None), location=Location(location=None)),
    df[df["Text"].str.find("Юрий Яковлев родился в Москве в 1928 году") > 0]["Text"].values[0]: Person(name=Name(first='юрий', last='яковлев'), date=Date(year=1928, month=None, day=None), location=Location(location='москва')),
    df[df["Text"].str.find("Смирнов родился 18 декабря 1937 года в городе Калининграде") > 0]["Text"].values[0]: Person(name=Name(first=None, last='смирнов'), date=Date(year=1937, month=12, day=18), location=Location(location=None)),
    df[df["Text"].str.find("Патрик родился в Бронксе в 1990 году") > 0]["Text"].values[0]: Person(name=Name(first='патрик', last=None), date=Date(year=1990, month=None, day=None), location=Location(location='бронкс')),
    df[df["Text"].str.find("Николай Караченцов родился 27 октября 1944 года в Москве") > 0]["Text"].values[0]: Person(name=Name(first='николай', last='караченцов'), date=Date(year=1944, month=10, day=27), location=Location(location='москва')),
    df[df["Text"].str.find("Орест I (он же Орест Федорович Карелин-Романишин-Русин) родился в 1950 году во Львове") > 0]["Text"].values[0]: Person(name=Name(first='орест', last=None), date=Date(year=1950, month=None, day=None), location=Location(location='львов')),
    df[df["Text"].str.find("Сергей Довлатов родился в 1941 году") > 0]["Text"].values[0]: Person(name=Name(first='сергей', last='довлатов'), date=Date(year=1941, month=None, day=None), location=Location(location=None)),
    df[df["Text"].str.find("Головко родился 24 ноября 1932 года в городе Бар") > 0]["Text"].values[0]: Person(name=Name(first=None, last='головко'), date=Date(year=1932, month=11, day=24), location=Location(location='бар')),
    df[df["Text"].str.find("Игорь Доценко родился в 1953 году в Хмельницкой области Украины") > 0]["Text"].values[0]: Person(name=Name(first='игорь', last='доценко'), date=Date(year=1953, month=None, day=None), location=Location(location='хмельницкая')),
    df[df["Text"].str.find("Энгельбарт родился в 1925 году в США") > 0]["Text"].values[0]: Person(name=Name(first='энгельбарт', last=None), date=Date(year=1925, month=None, day=None), location=Location(location='сша')),
}

In [307]:
for text, target in test_sample.items():
    for match in parser.findall(text):
        result = match.fact
        print(text), print(result), print("\n")

В Театре имени Вахтангова состоялось прощание с народным артистом СССР Юрием Яковлевым. Как сообщает ИТАР-ТАСС, церемония прошла без гражданской панихиды - традиционные речи было решено заменить траурной музыкой и записью стихов, прочитанных самим актером.Одним из первых с артистом простился художественный руководитель театра Римас Туминас. На церемонии прощания также присутствовали Людмила Максакова, Марианна Вертинская, Андрей Мягков, Василий Лановой, Вера Васильева, Сергей Юрский, Эммануил Виторган, Альберт Филозов и другие артисты. В конце прощания вся труппа Вахтанговского театра поднялась на сцену. Юрия Яковлева похоронят на Новодевичьем кладбище Москвы.Юрий Яковлев скончался 30 ноября на 86-м году жизни в больнице, куда попал после обморока. Его памяти была посвящена церемония вручения премии «Звезда театрала», которая проходила 2 декабря в Театре Вахтангова.Материалы по теме18:50 30 ноября 2013Последняя пристаньПамяти Юрия ЯковлеваМатериалы по теме17:29 30 ноября 2013Глаза княз

In [301]:
# for match in Parser(LOCATION).findall("Игорь Доценко родился в 1953 году в Хмельницкой области Украины"):
#         print(match.fact)

Location(location='хмельницкая')
